<a href="https://colab.research.google.com/github/jesus1080/CompuacionEvolutiva-/blob/master/Circuitob%C3%A1sico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Objetivo: obtner el circuito digital dependiendo de la tabla de verdad ingresada.

In [ ]:
!pip install deap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 6.0 MB/s eta 0:00:00


Importación de librerias

In [ ]:
import operator
import math
import random
import numpy
import itertools
from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ruta = '/content/drive/MyDrive/DataSet/multiplexor.txt'
# lee el archivo de texto

matriz = []
with open(ruta, 'r') as f:
    for linea in f:
        fila = [int(numero) for numero in linea.strip().split()]
        matriz.append(fila)


print(matriz)


[[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 0, 1, 1, 0], [0, 0, 0, 1, 0, 0, 0], [0, 0, 0, 1, 0, 1, 0], [0, 0, 0, 1, 1, 0, 0], [0, 0, 0, 1, 1, 1, 0], [0, 0, 1, 0, 0, 0, 1], [0, 0, 1, 0, 0, 1, 1], [0, 0, 1, 0, 1, 0, 1], [0, 0, 1, 0, 1, 1, 1], [0, 0, 1, 1, 0, 0, 1], [0, 0, 1, 1, 0, 1, 1], [0, 0, 1, 1, 1, 0, 1], [0, 0, 1, 1, 1, 1, 1], [0, 1, 0, 0, 0, 0, 0], [0, 1, 0, 0, 0, 1, 0], [0, 1, 0, 0, 1, 0, 0], [0, 1, 0, 0, 1, 1, 0], [0, 1, 0, 1, 0, 0, 1], [0, 1, 0, 1, 0, 1, 1], [0, 1, 0, 1, 1, 0, 1], [0, 1, 0, 1, 1, 1, 1], [0, 1, 1, 0, 0, 0, 0], [0, 1, 1, 0, 0, 1, 0], [0, 1, 1, 0, 1, 0, 0], [0, 1, 1, 0, 1, 1, 0], [0, 1, 1, 1, 0, 0, 1], [0, 1, 1, 1, 0, 1, 1], [0, 1, 1, 1, 1, 0, 1], [0, 1, 1, 1, 1, 1, 1], [1, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 0, 1, 0], [1, 0, 0, 0, 1, 0, 1], [1, 0, 0, 0, 1, 1, 1], [1, 0, 0, 1, 0, 0, 0], [1, 0, 0, 1, 0, 1, 0], [1, 0, 0, 1, 1, 0, 1], [1, 0, 0, 1, 1, 1, 1], [1, 0, 1, 0, 0, 0, 0], [1, 0, 1, 0, 0, 1, 0], [1, 0, 1, 0, 1, 0, 1], [1, 0, 1, 

Definir el conjunto
de funciones f que se van a usar, para generar
la expresión matemática o lógica que trata de
satisfacer la muestra finita de datos.

In [ ]:
pset = gp.PrimitiveSet("MAIN",6)
pset.addPrimitive(operator.and_, 2)
pset.addPrimitive(operator.or_, 2)
pset.addPrimitive(operator.not_,1)
pset.addTerminal(1)
pset.addTerminal(0)
pset.renameArguments(ARG0="a")
pset.renameArguments(ARG1="b")
pset.renameArguments(ARG2="c")
pset.renameArguments(ARG3="d")
pset.renameArguments(ARG4="e")
pset.renameArguments(ARG5="f")

Definir el conjunto de datos de prueba, para este  caso se define una tabla de verdad, muestra cómo responde la salida de un circuito lógico a las varias combinaciones en las entradas

In [ ]:
samples=numpy.array(matriz)

Se define la función de evaluación en este caso se utiliza error cuadrático medio
https://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.3.1463&rep=rep1&type=pdf

In [ ]:
def evalSymbReg(individual):
    func = toolbox.compile(expr=individual)
    eval=math.sqrt((numpy.sum([math.pow((func(*row[:6])-row[6]),2) for row in samples]))/len(samples))
    return eval,


Definir hiperpárametros

In [ ]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMin)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

toolbox.register("evaluate", evalSymbReg)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=0, max_=3)
toolbox.register('mutate', gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

/usr/local/lib/python3.9/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.9/dist-packages/deap/creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


Costruir el Algoritmo evolutivo (PG) y ejecutar

In [ ]:
def main():
    #random.seed(318)

    pop = toolbox.population(n=50)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)

    algorithms.eaSimple(pop, toolbox, 0.5, 0.3, 100, stats, halloffame=hof)

    return pop, stats, hof

if __name__ == "__main__":
    p,s,h=main()
    sol=h[0]
    print('mejor Solución encontrada',sol)
    print('longitud',len(sol))
    func = toolbox.compile(expr=sol)

    test=[func(*row[:6]) for row in samples]
    print(samples)
    print(test)



gen	nevals	avg     	std      	min     	max     
0  	50    	0.704312	0.0675953	0.586302	0.829156
1  	32    	0.659056	0.0560789	0.559017	0.790569
2  	32    	0.633664	0.0533408	0.559017	0.790569
3  	30    	0.623765	0.0528458	0.559017	0.790569
4  	33    	0.62031 	0.0577978	0.53033 	0.790569
5  	29    	0.586517	0.0362053	0.53033 	0.707107
6  	34    	0.584493	0.042493 	0.53033 	0.707107
7  	33    	0.571872	0.0374135	0.53033 	0.707107
8  	35    	0.564134	0.0454461	0.484123	0.707107
9  	26    	0.551504	0.0422046	0.484123	0.707107
10 	32    	0.543842	0.0416619	0.484123	0.707107
11 	33    	0.556692	0.059428 	0.484123	0.707107
12 	32    	0.542037	0.052525 	0.484123	0.728869
13 	34    	0.539945	0.0588164	0.484123	0.790569
14 	37    	0.533722	0.0598158	0.450694	0.790569
15 	38    	0.524996	0.04335  	0.450694	0.637377
16 	33    	0.520527	0.0498928	0.467707	0.728869
17 	33    	0.518201	0.052127 	0.395285	0.684653
18 	31    	0.520173	0.0710297	0.395285	0.71807 
19 	30    	0.485168	0.049237 	0.395285	0